#Project name

#1. Stop and Search Data
##1.1 Download stop and search data and combine into single file
The data comes from:
For the purpose of this project, we downloaded stop and search for the year 2019 -- Jan, 1 to Dec, 31. 
The source files were monthly reports - we combine them below. 

In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame as gdf
import glob 
import json
import numpy as np
import os

In [6]:
files = glob.glob("/Users/sean/desktop/proj_london/data/source_data/*.csv")
appended = [pd.read_csv(f) for f in files]
combined = pd.concat(appended, ignore_index=True)
combined.to_csv(r'/Users/sean/desktop/proj_london/data/combined_stop_data.csv')

##1.2 Read combined stop and search data 
Read combined data csv and check data:

In [ ]:
df = pd.read_csv('combined_stop_data.csv', dtype={"Latitude": str, "Longitude": str})
df.head()

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268469 entries, 0 to 268468
Data columns (total 16 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Unnamed: 0                                268469 non-null  int64  
 1   Type                                      268469 non-null  object 
 2   Date                                      268469 non-null  object 
 3   Part of a policing operation              265940 non-null  float64
 4   Policing operation                        0 non-null       float64
 5   Latitude                                  241403 non-null  object 
 6   Longitude                                 241403 non-null  object 
 7   Gender                                    264904 non-null  object 
 8   Age range                                 239826 non-null  object 
 9   Self-defined ethnicity                    264774 non-null  object 
 10  Officer-defined ethn

##1.3 Clean dataframe
Change column names and drop irrelevant columns:

In [71]:
df.rename(columns=
         {'Self-defined ethnicity': 'ethnicity',
         'Officer-defined ethnicity':'officer_ethnicity',
         'Object of search': 'purpose',
         'Part of a policing operation': 'operation',
         'Age range': 'age_range',
         'Gender': 'gender',
         'Outcome': 'outcome',
         'Legislation': 'legislation',
         'Date': 'date',
          'Latitude': 'latitude',
          'Longitude': 'longitude',
          'Type': 'type'
         },
         inplace=True)
df.columns

Index(['Unnamed: 0', 'type', 'date', 'operation', 'Policing operation',
       'latitude', 'longitude', 'gender', 'age_range', 'ethnicity',
       'officer_ethnicity', 'legislation', 'purpose', 'outcome',
       'Outcome linked to object of search',
       'Removal of more than just outer clothing'],
      dtype='object')

In [72]:
df = df.drop(['Policing operation', 'Outcome linked to object of search', 'Removal of more than just outer clothing'], axis = 1) 

In [73]:
df_cleaned = df.dropna(axis=0)

In [74]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212165 entries, 0 to 268315
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         212165 non-null  int64  
 1   type               212165 non-null  object 
 2   date               212165 non-null  object 
 3   operation          212165 non-null  float64
 4   latitude           212165 non-null  object 
 5   longitude          212165 non-null  object 
 6   gender             212165 non-null  object 
 7   age_range          212165 non-null  object 
 8   ethnicity          212165 non-null  object 
 9   officer_ethnicity  212165 non-null  object 
 10  legislation        212165 non-null  object 
 11  purpose            212165 non-null  object 
 12  outcome            212165 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 22.7+ MB


##1.4 Add coordinate column
Combine lat and long to create coordinate column for easy referencing

In [75]:
df_cleaned['coord'] = "(" + df_cleaned.latitude + ", " + df_cleaned.longitude + ")" 

<ipython-input-75-0f464366702c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['coord'] = "(" + df_cleaned.latitude + ", " + df_cleaned.longitude + ")"


In [76]:
df_cleaned.head()

,Unnamed: 0,type,date,operation,latitude,longitude,gender,age_range,ethnicity,officer_ethnicity,legislation,purpose,outcome,coord
0,0,Person and Vehicle search,2019-07-31T23:00:00+00:00,0.0,51.460887,0.005072,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.460887, 0.005072)"
1,1,Person and Vehicle search,2019-07-31T23:02:00+00:00,0.0,51.480146999999995,-0.507249,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,"(51.480146999999995, -0.507249)"
4,4,Person search,2019-07-31T23:05:00+00:00,0.0,51.486675,-0.093976,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.486675, -0.093976)"
5,5,Person search,2019-07-31T23:05:00+00:00,0.0,51.509962,-0.131179,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)"
8,8,Person search,2019-07-31T23:07:00+00:00,0.0,51.509962,-0.131179,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)"


In [77]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212165 entries, 0 to 268315
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         212165 non-null  int64  
 1   type               212165 non-null  object 
 2   date               212165 non-null  object 
 3   operation          212165 non-null  float64
 4   latitude           212165 non-null  object 
 5   longitude          212165 non-null  object 
 6   gender             212165 non-null  object 
 7   age_range          212165 non-null  object 
 8   ethnicity          212165 non-null  object 
 9   officer_ethnicity  212165 non-null  object 
 10  legislation        212165 non-null  object 
 11  purpose            212165 non-null  object 
 12  outcome            212165 non-null  object 
 13  coord              212165 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 24.3+ MB


##1.5.1 Clean up ethnicity column
There are many overlapping sub-categories. Group ethnicity together to eliminate said overlap.

In [78]:
df_cleaned['ethnicity'].value_counts()

White - English/Welsh/Scottish/Northern Irish/British                                   41346
Other ethnic group - Not stated                                                         34068
Black/African/Caribbean/Black British - Any other Black/African/Caribbean background    28721
White - Any other White background                                                      27215
Black/African/Caribbean/Black British - African                                         20187
Asian/Asian British - Any other Asian background                                        14911
Black/African/Caribbean/Black British - Caribbean                                       13425
Asian/Asian British - Bangladeshi                                                        9329
Other ethnic group - Any other ethnic group                                              5797
Asian/Asian British - Pakistani                                                          4033
Mixed/Multiple ethnic groups - Any other Mixed/Multiple ethn

In [79]:
def func(x):
    if 'White' in x:
        return 'White'
    elif 'Black' in x:
        return 'Black'
    elif 'Asian' in x:
        return 'Asian'
    elif 'Mixed' in x:
        return 'Mixed group'
    elif 'Not stated' in x:
        return 'Not stated'
    else:
        return 'Other'

In [80]:
df_cleaned['ethnicity_simple'] = df_cleaned['ethnicity'].apply(func)

<ipython-input-80-a36d89105834>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['ethnicity_simple'] = df_cleaned['ethnicity'].apply(func)


In [81]:
df_cleaned.head()

,Unnamed: 0,type,date,operation,latitude,longitude,gender,age_range,ethnicity,officer_ethnicity,legislation,purpose,outcome,coord,ethnicity_simple
0,0,Person and Vehicle search,2019-07-31T23:00:00+00:00,0.0,51.460887,0.005072,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.460887, 0.005072)",White
1,1,Person and Vehicle search,2019-07-31T23:02:00+00:00,0.0,51.480146999999995,-0.507249,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,"(51.480146999999995, -0.507249)",Black
4,4,Person search,2019-07-31T23:05:00+00:00,0.0,51.486675,-0.093976,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.486675, -0.093976)",Asian
5,5,Person search,2019-07-31T23:05:00+00:00,0.0,51.509962,-0.131179,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)",Not stated
8,8,Person search,2019-07-31T23:07:00+00:00,0.0,51.509962,-0.131179,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)",Black


In [82]:
df_cleaned['ethnicity_simple'].value_counts()

White          74017
Black          62333
Not stated     34068
Asian          32085
Other           5797
Mixed group     3865
Name: ethnicity_simple, dtype: int64

##1.5.2 Clean up officer ethnicity column

In [83]:
df_cleaned['officer_ethnicity'].value_counts()

White    83160
Black    82994
Asian    37488
Other     8523
Name: officer_ethnicity, dtype: int64

In [84]:
def func1(x):
    if 'White' in x:
        return 'White - officer'
    elif 'Black' in x:
        return 'Black - officer'
    elif 'Asian' in x:
        return 'Asian - officer'
    else:
        return 'Other - officer'

In [85]:
df_cleaned['ethnicity_officer'] = df_cleaned['officer_ethnicity'].apply(func1)

<ipython-input-85-2d4506b5d55c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['ethnicity_officer'] = df_cleaned['officer_ethnicity'].apply(func1)


In [86]:
df_cleaned.head()

,Unnamed: 0,type,date,operation,latitude,longitude,gender,age_range,ethnicity,officer_ethnicity,legislation,purpose,outcome,coord,ethnicity_simple,ethnicity_officer
0,0,Person and Vehicle search,2019-07-31T23:00:00+00:00,0.0,51.460887,0.005072,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.460887, 0.005072)",White,White - officer
1,1,Person and Vehicle search,2019-07-31T23:02:00+00:00,0.0,51.480146999999995,-0.507249,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,"(51.480146999999995, -0.507249)",Black,Black - officer
4,4,Person search,2019-07-31T23:05:00+00:00,0.0,51.486675,-0.093976,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.486675, -0.093976)",Asian,Asian - officer
5,5,Person search,2019-07-31T23:05:00+00:00,0.0,51.509962,-0.131179,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)",Not stated,White - officer
8,8,Person search,2019-07-31T23:07:00+00:00,0.0,51.509962,-0.131179,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)",Black,Asian - officer


##1.5.3 Clean up gender column

In [87]:
df_cleaned['gender'].value_counts()

Male      197801
Female     14303
Other         61
Name: gender, dtype: int64

In [88]:
def func2(x):
    if 'Other' in x:
        return 'Other gender'
    elif 'Male' in x:
        return 'Male'
    else:
        return 'Female'

In [89]:
df_cleaned['gender_clean'] = df_cleaned['gender'].apply(func2)

<ipython-input-89-c90c2028b422>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['gender_clean'] = df_cleaned['gender'].apply(func2)


In [90]:
df_cleaned['gender_clean'].value_counts()

Male            197801
Female           14303
Other gender        61
Name: gender_clean, dtype: int64

In [91]:
df_cleaned.to_csv('cleaned_stop_data.csv')

##1.6 Convert stop and search data GeoJSON

In [92]:
file = 'cleaned_stop_data.csv'
points = pd.read_csv(file)
points = pd.DataFrame(points)
points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212165 entries, 0 to 212164
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         212165 non-null  int64  
 1   Unnamed: 0.1       212165 non-null  int64  
 2   type               212165 non-null  object 
 3   date               212165 non-null  object 
 4   operation          212165 non-null  float64
 5   latitude           212165 non-null  float64
 6   longitude          212165 non-null  float64
 7   gender             212165 non-null  object 
 8   age_range          212165 non-null  object 
 9   ethnicity          212165 non-null  object 
 10  officer_ethnicity  212165 non-null  object 
 11  legislation        212165 non-null  object 
 12  purpose            212165 non-null  object 
 13  outcome            212165 non-null  object 
 14  coord              212165 non-null  object 
 15  ethnicity_simple   212165 non-null  object 
 16  et

In [93]:
points_df = gdf(points, geometry=gpd.points_from_xy(points.longitude, points.latitude))
points_df.head()

,Unnamed: 0,Unnamed: 0.1,type,date,operation,latitude,longitude,gender,age_range,ethnicity,officer_ethnicity,legislation,purpose,outcome,coord,ethnicity_simple,ethnicity_officer,gender_clean,geometry
0,0,0,Person and Vehicle search,2019-07-31T23:00:00+00:00,0.0,51.460887,0.005072,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.460887, 0.005072)",White,White - officer,Male,POINT (0.00507 51.46089)
1,1,1,Person and Vehicle search,2019-07-31T23:02:00+00:00,0.0,51.480147,-0.507249,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,"(51.480146999999995, -0.507249)",Black,Black - officer,Male,POINT (-0.50725 51.48015)
2,4,4,Person search,2019-07-31T23:05:00+00:00,0.0,51.486675,-0.093976,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.486675, -0.093976)",Asian,Asian - officer,Male,POINT (-0.09398 51.48667)
3,5,5,Person search,2019-07-31T23:05:00+00:00,0.0,51.509962,-0.131179,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)",Not stated,White - officer,Male,POINT (-0.13118 51.50996)
4,8,8,Person search,2019-07-31T23:07:00+00:00,0.0,51.509962,-0.131179,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)",Black,Asian - officer,Male,POINT (-0.13118 51.50996)


In [94]:
print(points_df.crs)

None


In [95]:
points_df.crs = {"init":"epsg:4326"}
print(points_df.crs)

{'init': 'epsg:4326'}


##1.6.1 Add borough data for each stop and search 
Add London borough data and determine where each stop and seach occured, and add that information to stop and search dataframe. 

In [96]:
borough_file = 'id_london.json'
boro_df = gpd.read_file(borough_file)

In [31]:
boro_df.head()

,id,name,code,area_hectares,inner_statistical,geometry
0,1,Kingston upon Thames,E09000021,3726.117,False,"MULTIPOLYGON (((-0.33068 51.32901, -0.33059 51..."
1,2,Croydon,E09000008,8649.441,False,"MULTIPOLYGON (((-0.06402 51.31864, -0.06408 51..."
2,3,Bromley,E09000006,15013.487,False,"MULTIPOLYGON (((0.01213 51.29960, 0.01196 51.2..."
3,4,Hounslow,E09000018,5658.541,False,"MULTIPOLYGON (((-0.24456 51.48870, -0.24468 51..."
4,5,Ealing,E09000009,5554.428,False,"MULTIPOLYGON (((-0.41183 51.53408, -0.41188 51..."


In [98]:
boro_df = boro_df.sort_values('name')
boro_df

,id,name,code,area_hectares,inner_statistical,geometry
31,32,Barking and Dagenham,E09000002,3779.934,False,"MULTIPOLYGON (((0.06971 51.54406, 0.07035 51.5..."
9,10,Barnet,E09000003,8674.837,False,"MULTIPOLYGON (((-0.19990 51.67017, -0.19970 51..."
14,15,Bexley,E09000004,6428.649,False,"MULTIPOLYGON (((0.12021 51.51144, 0.12159 51.5..."
8,9,Brent,E09000005,4323.270,False,"MULTIPOLYGON (((-0.19657 51.52765, -0.19685 51..."
2,3,Bromley,E09000006,15013.487,False,"MULTIPOLYGON (((0.01213 51.29960, 0.01196 51.2..."
25,26,Camden,E09000007,2178.932,True,"MULTIPOLYGON (((-0.14242 51.56912, -0.14250 51..."
32,33,City of London,E09000001,314.942,True,"MULTIPOLYGON (((-0.11157 51.51075, -0.11159 51..."
1,2,Croydon,E09000008,8649.441,False,"MULTIPOLYGON (((-0.06402 51.31864, -0.06408 51..."
4,5,Ealing,E09000009,5554.428,False,"MULTIPOLYGON (((-0.41183 51.53408, -0.41188 51..."
15,16,Enfield,E09000010,8220.025,False,"MULTIPOLYGON (((-0.10580 51.69187, -0.10557 51..."


In [99]:
df = gpd.sjoin(points_df, boro_df, how='left', op='within')

In [100]:
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,Unnamed: 0.1,type,date,operation,latitude,longitude,gender,age_range,ethnicity,officer_ethnicity,legislation,purpose,outcome,coord,ethnicity_simple,ethnicity_officer,gender_clean,geometry,index_right,id,name,code,area_hectares,inner_statistical
0,0,0,Person and Vehicle search,2019-07-31T23:00:00+00:00,0.0,51.460887,0.005072,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.460887, 0.005072)",White,White - officer,Male,POINT (0.00507 51.46089),12.0,13.0,Lewisham,E09000023,3531.706,True
1,1,1,Person and Vehicle search,2019-07-31T23:02:00+00:00,0.0,51.480147,-0.507249,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,"(51.480146999999995, -0.507249)",Black,Black - officer,Male,POINT (-0.50725 51.48015),NaN,NaN,NaN,NaN,NaN,NaN
2,4,4,Person search,2019-07-31T23:05:00+00:00,0.0,51.486675,-0.093976,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,"(51.486675, -0.093976)",Asian,Asian - officer,Male,POINT (-0.09398 51.48667),11.0,12.0,Southwark,E09000028,2991.340,True
3,5,5,Person search,2019-07-31T23:05:00+00:00,0.0,51.509962,-0.131179,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)",Not stated,White - officer,Male,POINT (-0.13118 51.50996),24.0,25.0,Westminster,E09000033,2203.005,True
4,8,8,Person search,2019-07-31T23:07:00+00:00,0.0,51.509962,-0.131179,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,"(51.509962, -0.131179)",Black,Asian - officer,Male,POINT (-0.13118 51.50996),24.0,25.0,Westminster,E09000033,2203.005,True


In [101]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 212165 entries, 0 to 212164
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Unnamed: 0         212165 non-null  int64   
 1   Unnamed: 0.1       212165 non-null  int64   
 2   type               212165 non-null  object  
 3   date               212165 non-null  object  
 4   operation          212165 non-null  float64 
 5   latitude           212165 non-null  float64 
 6   longitude          212165 non-null  float64 
 7   gender             212165 non-null  object  
 8   age_range          212165 non-null  object  
 9   ethnicity          212165 non-null  object  
 10  officer_ethnicity  212165 non-null  object  
 11  legislation        212165 non-null  object  
 12  purpose            212165 non-null  object  
 13  outcome            212165 non-null  object  
 14  coord              212165 non-null  object  
 15  ethnicity_simple   212165 

##1.6.2 Trim dataframe to only columns to export to JSON
Also change column names to appropriately

In [102]:
use = ['type', 'date', 'gender_clean', 'age_range', 'ethnicity', 'ethnicity_simple', 'purpose', 'legislation', 'outcome', 'name','ethnicity_officer', 'geometry']
df_toJson = df[use]
df_toJson.head()

,type,date,gender_clean,age_range,ethnicity,ethnicity_simple,purpose,legislation,outcome,name,ethnicity_officer,geometry
0,Person and Vehicle search,2019-07-31T23:00:00+00:00,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Controlled drugs,Misuse of Drugs Act 1971 (section 23),A no further action disposal,Lewisham,White - officer,POINT (0.00507 51.46089)
1,Person and Vehicle search,2019-07-31T23:02:00+00:00,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Arrest,NaN,Black - officer,POINT (-0.50725 51.48015)
2,Person search,2019-07-31T23:05:00+00:00,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Controlled drugs,Misuse of Drugs Act 1971 (section 23),A no further action disposal,Southwark,Asian - officer,POINT (-0.09398 51.48667)
3,Person search,2019-07-31T23:05:00+00:00,Male,10-17,Other ethnic group - Not stated,Not stated,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Community resolution,Westminster,White - officer,POINT (-0.13118 51.50996)
4,Person search,2019-07-31T23:07:00+00:00,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Black,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Community resolution,Westminster,Asian - officer,POINT (-0.13118 51.50996)


In [103]:
df_toJson.rename({'gender_clean': 'gender'}, axis=1, inplace=True)
df_toJson.rename({'ethnicity': 'old_ethnicity'}, axis=1, inplace=True)

/opt/anaconda3/envs/london-data/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [104]:
df_toJson.rename({'ethnicity_simple': 'ethnicity'}, axis=1, inplace=True)
df_toJson.rename({'ethnicity_officer': 'officer_ethnicity'}, axis=1, inplace=True)
df_toJson.head()

,type,date,gender,age_range,old_ethnicity,ethnicity,purpose,legislation,outcome,name,officer_ethnicity,geometry
0,Person and Vehicle search,2019-07-31T23:00:00+00:00,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Controlled drugs,Misuse of Drugs Act 1971 (section 23),A no further action disposal,Lewisham,White - officer,POINT (0.00507 51.46089)
1,Person and Vehicle search,2019-07-31T23:02:00+00:00,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Arrest,NaN,Black - officer,POINT (-0.50725 51.48015)
2,Person search,2019-07-31T23:05:00+00:00,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Controlled drugs,Misuse of Drugs Act 1971 (section 23),A no further action disposal,Southwark,Asian - officer,POINT (-0.09398 51.48667)
3,Person search,2019-07-31T23:05:00+00:00,Male,10-17,Other ethnic group - Not stated,Not stated,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Community resolution,Westminster,White - officer,POINT (-0.13118 51.50996)
4,Person search,2019-07-31T23:07:00+00:00,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Black,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Community resolution,Westminster,Asian - officer,POINT (-0.13118 51.50996)


In [105]:
df_toJson.to_file("points.geojson", driver='GeoJSON')

#2 Reshape London GeoJSON
##2.1 Add total stop and searches per borough 
Download London borough GeoJSON and for each borough add property "total" that tallies the total number of stops per borough.

In [106]:
boroughs = pd.DataFrame(df_toJson.name.value_counts().reset_index().values, columns=["name", "total"])
borough_total = boroughs.sort_index(axis = 0, ascending=True)

In [107]:
borough_total.head()

,name,total
0,Westminster,15165
1,Newham,14008
2,Southwark,13088
3,Tower Hamlets,12891
4,Lambeth,11973


In [108]:
merged = pd.merge(boro_df, borough_total, left_on='name', right_on='name', how='left')
merged_df = pd.DataFrame(data=merged)

In [109]:
merged_df.head()

,id,name,code,area_hectares,inner_statistical,geometry,total
0,32,Barking and Dagenham,E09000002,3779.934,False,"MULTIPOLYGON (((0.06971 51.54406, 0.07035 51.5...",4225
1,10,Barnet,E09000003,8674.837,False,"MULTIPOLYGON (((-0.19990 51.67017, -0.19970 51...",2661
2,15,Bexley,E09000004,6428.649,False,"MULTIPOLYGON (((0.12021 51.51144, 0.12159 51.5...",2903
3,9,Brent,E09000005,4323.270,False,"MULTIPOLYGON (((-0.19657 51.52765, -0.19685 51...",7235
4,3,Bromley,E09000006,15013.487,False,"MULTIPOLYGON (((0.01213 51.29960, 0.01196 51.2...",5795


In [110]:
merged_df['total'] = pd.to_numeric(merged_df['total'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 33 non-null     int64   
 1   name               33 non-null     object  
 2   code               33 non-null     object  
 3   area_hectares      33 non-null     float64 
 4   inner_statistical  33 non-null     bool    
 5   geometry           33 non-null     geometry
 6   total              33 non-null     int64   
dtypes: bool(1), float64(1), geometry(1), int64(2), object(2)
memory usage: 1.8+ KB


In [111]:
geo_frame = gdf(merged_df, crs="EPSG:4326", geometry=merged_df['geometry'])
geo_frame.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 33 non-null     int64   
 1   name               33 non-null     object  
 2   code               33 non-null     object  
 3   area_hectares      33 non-null     float64 
 4   inner_statistical  33 non-null     bool    
 5   geometry           33 non-null     geometry
 6   total              33 non-null     int64   
dtypes: bool(1), float64(1), geometry(1), int64(2), object(2)
memory usage: 1.8+ KB


In [112]:
df_toJson.head()

,type,date,gender,age_range,old_ethnicity,ethnicity,purpose,legislation,outcome,name,officer_ethnicity,geometry
0,Person and Vehicle search,2019-07-31T23:00:00+00:00,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Controlled drugs,Misuse of Drugs Act 1971 (section 23),A no further action disposal,Lewisham,White - officer,POINT (0.00507 51.46089)
1,Person and Vehicle search,2019-07-31T23:02:00+00:00,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Arrest,NaN,Black - officer,POINT (-0.50725 51.48015)
2,Person search,2019-07-31T23:05:00+00:00,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Controlled drugs,Misuse of Drugs Act 1971 (section 23),A no further action disposal,Southwark,Asian - officer,POINT (-0.09398 51.48667)
3,Person search,2019-07-31T23:05:00+00:00,Male,10-17,Other ethnic group - Not stated,Not stated,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Community resolution,Westminster,White - officer,POINT (-0.13118 51.50996)
4,Person search,2019-07-31T23:07:00+00:00,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Black,Controlled drugs,Misuse of Drugs Act 1971 (section 23),Community resolution,Westminster,Asian - officer,POINT (-0.13118 51.50996)


##2.2 Get age range totals
Get total of age range per borough

In [113]:
age_totals = pd.DataFrame(df_toJson.pivot_table(index=['name'], columns=['age_range'], aggfunc='size', fill_value=0)).reset_index().rename_axis(None,axis=1)
age_totals.head()

,name,10-17,18-24,25-34,over 34,under 10
0,Barking and Dagenham,969,1558,1090,606,2
1,Barnet,645,1026,572,417,1
2,Bexley,842,1071,601,388,1
3,Brent,1369,3148,1680,1038,0
4,Bromley,1686,1981,1078,1048,2


##2.3 Get gender totals
Get total number of stop and searches per gender per borough

In [114]:
gender_totals = pd.DataFrame(df_toJson.pivot_table(index=['name'], columns=['gender'], aggfunc='size', fill_value=0)).reset_index().rename_axis(None,axis=1)
gender_totals.head() 

,name,Female,Male,Other gender
0,Barking and Dagenham,277,3947,1
1,Barnet,178,2481,2
2,Bexley,227,2675,1
3,Brent,375,6859,1
4,Bromley,708,5085,2


##2.4 Get ethnicity totals
Get total number of stop and searches per ethnicity per borough

In [115]:
ethnicity_totals = pd.DataFrame(df_toJson.pivot_table(index=['name'], columns=['ethnicity'], aggfunc='size', fill_value=0)).reset_index().rename_axis(None,axis=1)
ethnicity_totals.head()

,name,Asian,Black,Mixed group,Not stated,Other,White
0,Barking and Dagenham,755,1072,69,272,101,1956
1,Barnet,304,630,85,438,132,1072
2,Bexley,95,432,38,210,30,2098
3,Brent,1023,2872,151,1283,254,1652
4,Bromley,166,1201,98,551,82,3697


##2.5 Get officer ethnicity totals
Get total number of stop and searches per officer ethnicity per borough

In [116]:
officer_totals = pd.DataFrame(df_toJson.pivot_table(index=['name'], columns=['officer_ethnicity'], aggfunc='size', fill_value=0)).reset_index().rename_axis(None,axis=1)
officer_totals.head()

,name,Asian - officer,Black - officer,Other - officer,White - officer
0,Barking and Dagenham,845,1244,38,2098
1,Barnet,337,832,286,1206
2,Bexley,132,542,35,2194
3,Brent,1190,3692,494,1859
4,Bromley,235,1580,59,3921


##2.6 Get purpose totals
Get total number of stop and searches per purpose of stops per borough

In [117]:
purpose_totals = pd.DataFrame(df_toJson.pivot_table(index=['name'], columns=['purpose'], aggfunc='size', fill_value=0)).reset_index().rename_axis(None,axis=1)
purpose_totals.head()

,name,Anything to threaten or harm anyone,Articles for use in criminal damage,Controlled drugs,Evidence of offences under the Act,Firearms,Fireworks,Offensive weapons,Stolen goods
0,Barking and Dagenham,106,11,2664,200,26,20,664,534
1,Barnet,32,4,1461,273,24,16,437,414
2,Bexley,71,26,1724,289,9,34,356,394
3,Brent,264,19,4529,324,51,18,1348,682
4,Bromley,91,97,2808,654,22,16,1014,1093


##2.7 Get type totals
Get total number of stop and searches per type of stops per borough

In [118]:
type_totals = pd.DataFrame(df_toJson.pivot_table(index=['name'], columns=['type'], aggfunc='size', fill_value=0)).reset_index().rename_axis(None,axis=1)
type_totals.head()

,name,Person and Vehicle search,Person search,Vehicle search
0,Barking and Dagenham,1076,3134,15
1,Barnet,740,1917,4
2,Bexley,712,2185,6
3,Brent,1931,5283,21
4,Bromley,1120,4665,10


##2.8 Add all totals to London GeoJSON

In [119]:
two = type_totals.merge(purpose_totals, on='name')
three = two.merge(officer_totals, on='name')
four = three.merge(ethnicity_totals, on='name')
five = four.merge(gender_totals, on='name')
six = five.merge(age_totals, on='name')
totals_df = six.merge(merged_df, on='name')
totals_df.head()

,name,Person and Vehicle search,Person search,Vehicle search,Anything to threaten or harm anyone,Articles for use in criminal damage,Controlled drugs,Evidence of offences under the Act,Firearms,Fireworks,Offensive weapons,Stolen goods,Asian - officer,Black - officer,Other - officer,White - officer,Asian,Black,Mixed group,Not stated,Other,White,Female,Male,Other gender,10-17,18-24,25-34,over 34,under 10,id,code,area_hectares,inner_statistical,geometry,total
0,Barking and Dagenham,1076,3134,15,106,11,2664,200,26,20,664,534,845,1244,38,2098,755,1072,69,272,101,1956,277,3947,1,969,1558,1090,606,2,32,E09000002,3779.934,False,"MULTIPOLYGON (((0.06971 51.54406, 0.07035 51.5...",4225
1,Barnet,740,1917,4,32,4,1461,273,24,16,437,414,337,832,286,1206,304,630,85,438,132,1072,178,2481,2,645,1026,572,417,1,10,E09000003,8674.837,False,"MULTIPOLYGON (((-0.19990 51.67017, -0.19970 51...",2661
2,Bexley,712,2185,6,71,26,1724,289,9,34,356,394,132,542,35,2194,95,432,38,210,30,2098,227,2675,1,842,1071,601,388,1,15,E09000004,6428.649,False,"MULTIPOLYGON (((0.12021 51.51144, 0.12159 51.5...",2903
3,Brent,1931,5283,21,264,19,4529,324,51,18,1348,682,1190,3692,494,1859,1023,2872,151,1283,254,1652,375,6859,1,1369,3148,1680,1038,0,9,E09000005,4323.270,False,"MULTIPOLYGON (((-0.19657 51.52765, -0.19685 51...",7235
4,Bromley,1120,4665,10,91,97,2808,654,22,16,1014,1093,235,1580,59,3921,166,1201,98,551,82,3697,708,5085,2,1686,1981,1078,1048,2,3,E09000006,15013.487,False,"MULTIPOLYGON (((0.01213 51.29960, 0.01196 51.2...",5795


In [120]:
totals_geo = gdf(totals_df, crs="EPSG:4326")
totals_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 33 entries, 0 to 32
Data columns (total 36 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   name                                 33 non-null     object  
 1   Person and Vehicle search            33 non-null     int64   
 2   Person search                        33 non-null     int64   
 3   Vehicle search                       33 non-null     int64   
 4   Anything to threaten or harm anyone  33 non-null     int64   
 5   Articles for use in criminal damage  33 non-null     int64   
 6   Controlled drugs                     33 non-null     int64   
 7   Evidence of offences under the Act   33 non-null     int64   
 8   Firearms                             33 non-null     int64   
 9   Fireworks                            33 non-null     int64   
 10  Offensive weapons                    33 non-null     int64   
 11  Stolen goods 

In [121]:
totals_geo.head()

,name,Person and Vehicle search,Person search,Vehicle search,Anything to threaten or harm anyone,Articles for use in criminal damage,Controlled drugs,Evidence of offences under the Act,Firearms,Fireworks,Offensive weapons,Stolen goods,Asian - officer,Black - officer,Other - officer,White - officer,Asian,Black,Mixed group,Not stated,Other,White,Female,Male,Other gender,10-17,18-24,25-34,over 34,under 10,id,code,area_hectares,inner_statistical,geometry,total
0,Barking and Dagenham,1076,3134,15,106,11,2664,200,26,20,664,534,845,1244,38,2098,755,1072,69,272,101,1956,277,3947,1,969,1558,1090,606,2,32,E09000002,3779.934,False,"MULTIPOLYGON (((0.06971 51.54406, 0.07035 51.5...",4225
1,Barnet,740,1917,4,32,4,1461,273,24,16,437,414,337,832,286,1206,304,630,85,438,132,1072,178,2481,2,645,1026,572,417,1,10,E09000003,8674.837,False,"MULTIPOLYGON (((-0.19990 51.67017, -0.19970 51...",2661
2,Bexley,712,2185,6,71,26,1724,289,9,34,356,394,132,542,35,2194,95,432,38,210,30,2098,227,2675,1,842,1071,601,388,1,15,E09000004,6428.649,False,"MULTIPOLYGON (((0.12021 51.51144, 0.12159 51.5...",2903
3,Brent,1931,5283,21,264,19,4529,324,51,18,1348,682,1190,3692,494,1859,1023,2872,151,1283,254,1652,375,6859,1,1369,3148,1680,1038,0,9,E09000005,4323.270,False,"MULTIPOLYGON (((-0.19657 51.52765, -0.19685 51...",7235
4,Bromley,1120,4665,10,91,97,2808,654,22,16,1014,1093,235,1580,59,3921,166,1201,98,551,82,3697,708,5085,2,1686,1981,1078,1048,2,3,E09000006,15013.487,False,"MULTIPOLYGON (((0.01213 51.29960, 0.01196 51.2...",5795


In [122]:
boro_df

,id,name,code,area_hectares,inner_statistical,geometry
31,32,Barking and Dagenham,E09000002,3779.934,False,"MULTIPOLYGON (((0.06971 51.54406, 0.07035 51.5..."
9,10,Barnet,E09000003,8674.837,False,"MULTIPOLYGON (((-0.19990 51.67017, -0.19970 51..."
14,15,Bexley,E09000004,6428.649,False,"MULTIPOLYGON (((0.12021 51.51144, 0.12159 51.5..."
8,9,Brent,E09000005,4323.270,False,"MULTIPOLYGON (((-0.19657 51.52765, -0.19685 51..."
2,3,Bromley,E09000006,15013.487,False,"MULTIPOLYGON (((0.01213 51.29960, 0.01196 51.2..."
25,26,Camden,E09000007,2178.932,True,"MULTIPOLYGON (((-0.14242 51.56912, -0.14250 51..."
32,33,City of London,E09000001,314.942,True,"MULTIPOLYGON (((-0.11157 51.51075, -0.11159 51..."
1,2,Croydon,E09000008,8649.441,False,"MULTIPOLYGON (((-0.06402 51.31864, -0.06408 51..."
4,5,Ealing,E09000009,5554.428,False,"MULTIPOLYGON (((-0.41183 51.53408, -0.41188 51..."
15,16,Enfield,E09000010,8220.025,False,"MULTIPOLYGON (((-0.10580 51.69187, -0.10557 51..."


In [123]:
totals_geo.to_file("london.geojson", driver='GeoJSON')